# Blogs related to this exercise:

[Image Alignment (ECC) in OpenCV](https://www.learnopencv.com/image-alignment-ecc-in-opencv-c-python/)

[Image Alignment (Feature Based) using OpenCV](https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/)

# Image alignment with Enhanced Correlation Coefficient Maximization

In [4]:
import cv2
import numpy as np

# Read in the images to be aligned
img1 = cv2.imread("images/image1.jpg")
img2 = cv2.imread("images/image2.jpg")

# Convert images to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
#print(img2_gray.shape)

# Fine the size of image1
size = img1.shape

# Define the motion mode
warp_mode = cv2.MOTION_TRANSLATION

# Define 2x3 or 3x3 matrices and initialize the matrices to indentity
# A 2x3 matrix corresponding to Affine transformation
# A 3x3 matrix corresponding to Homography transformation
if warp_mode == cv2.MOTION_HOMOGRAPHY:
    warp_matrix = np.eye(3, 3, dtype=np.float32) # Has to be a float matrix
else:
    warp_matrix = np.eye(2, 3, dtype=np.float32) # Has to be a float matrix

# Specify the number of iterations
number_of_iterations = 5000
# Specify the threshold of the increment in the correlation coefficient between two iterations
termination_eps = 1e-10

# Define termination criteria
criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, number_of_iterations, termination_eps)

# Run the ecc algorithm, the desired warp parameters are stored in warp_matrix
retval, warp_matrix= cv2.findTransformECC(img1_gray, img2_gray, warp_matrix, warp_mode, criteria)

if warp_mode == cv2.MOTION_HOMOGRAPHY:
    # Use warpPerspective for Homography
    img2_aligned = cv2.warpPerspective(img2, warp_matrix, (size[1], size[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
else:
    # Use warpAffine for Translation, Euclidean and Affine
    img2_aligned = cv2.warpAffine(img2, warp_matrix, (size[1], size[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

# Show final results
cv2.imshow("Image 1", img1)
cv2.imshow("Image 2", img2)
cv2.imshow("Aligned Image2", img2_aligned)
cv2.waitKey(0)

-1

# Channel-wise alignment utilizing gradient

In [6]:
import cv2
import numpy as np

def get_gradient(img):
    # Calculate the x and y gradients using Sobel operator
    # cv2_35F corresponding with destination depth of 5
    grad_x = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3) 
    grad_y = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
    # Combine the two gradients
    grad = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)
    return grad

# Read 8-bit color image
# This is an image in which the three channels are concatenate vertically

img = cv2.imread("images/girls.jpg", cv2.IMREAD_GRAYSCALE)

# Find the width and height of the color image
size = img.shape
print(size)
height = size[0] // 3
width = size[1]

# Extract the three channels from the gray scale image
# and merge the three channels into one color image
img_colored = np.zeros((height, width, 3), dtype=np.uint8)
for i in range(0, 3):
    img_colored[:, :, i] = img[i*height: (i+1)*height, :]

# Allocate space for aligned image
img_aligned = np.zeros((height, width, 3), dtype=np.uint8)

# The blue and green channels will be aligned to the red channel
# so the red channel are copied as is
img_aligned[:, :, 2] = img_colored[:, :, 2]

# Define motion model
warp_mode = cv2.MOTION_HOMOGRAPHY

# Set the warp matrix to identity
if warp_mode == cv2.MOTION_HOMOGRAPHY:
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else:
    warp_matrix = np.eye(2, 3, dtype=np.folat32)
    
# Set the stopping criteria for the algorithm
criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 5000, 1e-10)

# Warp the blue and green channels to the red channel
for i in range(0, 2):
    # The three channels are not strongly correlated, so you need the gradient instead
    retval, warp_matrix = cv2.findTransformECC(get_gradient(img_colored[:, :, 2]), get_gradient(img_colored[:, :, i]), warp_matrix, warp_mode, criteria)
    
    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        # Use warpPerspective when transformation is homography
        img_aligned[:, :, i] = cv2.warpPerspective(img_colored[:, :, i], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else:
        # Use warpAffine when transformation is not homography
        img_aligned[:, :, i] = cv2.warpAffine(img_colored[:, :, i], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    print(warp_matrix)   

# Show final output
cv2.imshow("Color Image", img_colored)
cv2.imshow("Aligned Image", img_aligned)
cv2.waitKey(0)

(1941, 750)
[[  1.03404832e+00   2.44795810e-03  -2.74030662e+00]
 [  1.44800860e-02   1.02576125e+00  -5.16192245e+00]
 [  4.44452198e-05   5.57811109e-06   1.00000000e+00]]
[[  1.01441336e+00   5.32015401e-04  -6.62373006e-01]
 [  7.65844947e-03   1.00793850e+00   2.53184557e+00]
 [  1.95356297e-05   2.78898278e-06   1.00000000e+00]]


-1

<br>
<figure>
  <img src = "images/girls-aligned.jpg" width = "100%" style = "border: thin silver solid; padding: 1px">
      <figcaption style = "text-align:left; font-style:italic">Image alignment using color channels' gradient</figcaption>
</figure> 
<br>

# Feature based image alignment

In [11]:
import cv2
import numpy as np

MAX_MATCHES = 500
GOOD_MATCH_PERCENT = 0.15

def align_images(img1, img2):
    
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Detect ORB features and compute descriptors
    orb = cv2.ORB_create(MAX_MATCHES)
    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)
    
    # Match features
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    
    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
    
    # Remove matches that are not good enough
    num_good_matches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:num_good_matches]
    
    # Draw top matches
    img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", img_matches)
    
    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
    
    for i, match in enumerate(matches):
        # Here the 'queryIdx' and 'trainIdx' is really elusive to me
        # https://stackoverflow.com/questions/10765066/what-is-query-and-train-in-opencv-features2d
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
        
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
    
    # Use homography
    height, width, channels = img2.shape
    img1_aligned = cv2.warpPerspective(img1, h, (width, height))
    
    return img1_aligned, h

    
# Read in the reference image
reference_name = 'images/form.jpg'
print("Reading reference image:", reference_name)
reference_img = cv2.imread(reference_name, cv2.IMREAD_COLOR)

# Read in image to be aligned
tilted_name = "images/scanned-form.jpg"
print("Reading image to be aligned:", tilted_name)
tilted_img = cv2.imread(tilted_name, cv2.IMREAD_COLOR)

print("Aligning Images ...")
# Aligned image will be stored in aligned
aligned, h = align_images(tilted_img, reference_img)

# Write aligned image to disk
out_filename = 'images/aligned.jpg'
print("Saving aligned image:", out_filename)
cv2.imwrite(out_filename, aligned)

# Print estimated homography
print("Estimated homography:\n", h)

Reading reference image: images/form.jpg
Reading image to be aligned: images/scanned-form.jpg
Aligning Images ...
75
Saving aligned image: images/aligned.jpg
Estimated homography:
 [[  1.38140525e+00  -2.03885205e-01  -5.41252115e+00]
 [  2.17958789e-01   1.49648830e+00  -4.73234516e+02]
 [ -7.16127408e-05   1.04388020e-04   1.00000000e+00]]


<br>
<figure>
  <img src = "images/image-alignment-using-opencv.jpg" width = "100%" style = "border: thin silver solid; padding: 1px">
      <figcaption style = "text-align:left; font-style:italic">Image alignment with features</figcaption>
</figure> 
<br>